<a href="https://colab.research.google.com/github/IamJamesRooke/ztm_ai_agents/blob/master/ZTM_Capstone_Partnership_Lead_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# !pip install 'crewai[tools]' -q
# !pip install youtube-transcript-api -q

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [ ]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool
from pydantic import BaseModel

In [ ]:
llm = ChatOpenAI(model_name='gpt-4o', temperature=0)

# Data Inputs

In [ ]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [ ]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

# Research

In [ ]:
# Build Research Agent
research_agent = Agent(
    role="YouTube Influencer Researcher",
    goal =
    f"""
    Research the YouTube Channel {youtube_lead['name']} on url {youtube_lead['channel_url']}
    to assess suitability with {brand_info['name']}'s brand values: {brand_info['brand_values']}.
    """,
    backstory= f"You are an expert in Influencer partnerships with channels with these ideal characteristics: {brand_info['ideal_partner']}.",
    llm =llm,
    tools = [
        SerperDevTool(),
        ScrapeWebsiteTool(),
        YoutubeVideoSearchTool(),
        YoutubeChannelSearchTool(),
    ]
)

In [ ]:
# Define the channel research task.
channel_research_task = Task(
    description = f"""
    Research the chanel {youtube_lead['name']} on url {youtube_lead['channel_url']}:
    1. Identify channel and content type.
    2. Determine approximate subscriber count and engagement.
    3. Chack for audience alignment with {brand_info['name']}.
    """,
    expected_output = "A research report on the channel's key atritbutes and audience.",
    agent = research_agent
)

In [ ]:
# Define the videos  research task.
videos_research_task = Task(
    description = f"""
    Research the most popular videos from {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Summarize their topics and analyze their audienes.
    """,
    expected_output = "A research report on the most popular videos and their topics.",
    agent = research_agent,
)


In [ ]:
# Define a sentiment on the channel ia Google Searches.
sentiment_research_task = Task(
    description = f"""
    Conduct a sentiment analysis, via Google Search, of {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Look for user feedback, reviews, or discussions that indicate general sentiment.
    Search for instances on Reddit, being aware of their leftwing bias
    """,
    expected_output = "A sentiment analysis report on the channel.",
    agent = research_agent,
)

# AI Scoring Agent

In [ ]:
# Developing the scoring agent.
scoring_agent = Agent(
    role = "Partnership Scoring Agent",
    goal = f"Score the partnership between {brand_info['name']} and {youtube_lead['name']} on the following criteria: {brand_info['brand_values']}",
    backstory = f"You are an expert in Influencer Marketing working for {brand_info['name']}",
    llm = llm,
)

In [ ]:
# Create a base model task to provide a score
class Score(BaseModel):
  score: int
  justification: str

In [ ]:
# Define the scoring task.
scoring_task = Task(
    description= f"Score the partnership between {brand_info['name']} and {youtube_lead['name']}",
    expected_output= "A score between 1 and 10 with a justification explaining what fits and what doesn't.",
    agent=scoring_agent,
    context = [channel_research_task, videos_research_task, sentiment_research_task],
    output_pydantic = Score,
)

# Build AI Crew

In [ ]:
# Assemble AI Crew
crew = Crew(
    agents = [research_agent, scoring_agent],
    tasks = [
        channel_research_task,
        videos_research_task,
        sentiment_research_task,
        scoring_task],
    process = Process.sequential,
    verbose= False,
)

In [ ]:
# Activate
result = crew.kickoff()

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Thought:...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='\nThough...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [

# Outputs

In [ ]:
channel_research_output = result.tasks_output[0].raw
videos_research_output = result.tasks_output[1].raw
sentiment_research_output = result.tasks_output[2].raw
scoring_output = result.tasks_output[3].raw

In [ ]:
print("Channel Research Output:")
display(Markdown(channel_research_output))

Channel Research Output:


**Research Report on Theo - t3․gg YouTube Channel**

1. **Channel and Content Type:**
   - The channel "Theo - t3․gg" is focused on software development, particularly full-stack TypeScript. It is run by Theo, who is known for his work with the T3 stack. The content includes tech stack breakdowns, startup stories, and discussions on software development topics.

2. **Subscriber Count and Engagement:**
   - As of the latest data, the channel has approximately 441,000 subscribers and has accumulated over 71.47 million video views in total. The channel features videos with high view counts, indicating strong engagement from its audience.

3. **Audience Alignment with Zero to Mastery:**
   - The channel's audience consists of motivated learners interested in programming, software engineering, and tech career advice, aligning well with Zero to Mastery's brand values of quality education, community-driven growth, and career-focused learning. The content is up-to-date with current tech stacks, which is crucial for Zero to Mastery's focus on modern technology education.

Overall, the "Theo - t3․gg" YouTube channel appears to be a suitable partner for Zero to Mastery, given its focus on relevant tech topics and its engaged audience of learners.

In [ ]:
print("Videos Research Output:")
display(Markdown(videos_research_output))

Videos Research Output:


The most popular videos on the Theo - t3․gg YouTube channel cover a variety of topics relevant to software development and tech careers. These include discussions on industry trends, cost management in cloud services, critiques of popular technologies, and personal insights into startup challenges. The channel's content appeals to an audience of motivated learners and professionals, aligning well with Zero to Mastery's focus on quality education and career development in the tech industry.

In [ ]:
print("Sentiment Research Output:")
display(Markdown(sentiment_research_output))

Sentiment Research Output:


**Sentiment Analysis Report on Theo - t3․gg YouTube Channel**

1. **Positive Sentiments:**
   - Theo - t3․gg is recognized for providing valuable content on full-stack TypeScript and software development. Many users appreciate the technical insights and the educational value of the content.
   - The channel is noted for its strong engagement, with high view counts and a substantial subscriber base, indicating a positive reception from a large audience.

2. **Negative Sentiments:**
   - Some users on Reddit have expressed concerns about Theo's personality, describing it as off-putting or narcissistic. This sentiment suggests that while the content is valuable, the delivery or personal style may not resonate with everyone.
   - There are mentions of communication struggles, which some viewers find makes the content harder to digest.

3. **Mixed Sentiments:**
   - While some viewers appreciate Theo's strong opinions on technology, others feel that these opinions can be polarizing. This indicates a mixed reception where some viewers value the honesty, while others may find it too blunt.

Overall, the sentiment towards Theo - t3․gg is generally positive in terms of content quality and educational value, aligning well with Zero to Mastery's brand values. However, there are some personality and communication style concerns that may affect viewer perception.

In [ ]:
# Format the output using Markdown
markdown_output = f"""
**Partnership Score:** {scoring_data['score']}/10

**Justification:** {scoring_data['justification']}
"""

display(Markdown(markdown_output))


**Partnership Score:** 8/10

**Justification:** The partnership between Zero to Mastery and Theo - t3․gg scores an 8 out of 10. The channel aligns well with Zero to Mastery's focus on quality education, community-driven growth, and career-focused learning. Theo's content is up-to-date with current tech stacks, particularly full-stack TypeScript, which is crucial for modern technology education. The channel's large and engaged audience of motivated learners interested in programming and tech careers further supports this alignment. However, the score is slightly reduced due to some negative sentiments regarding Theo's personality and communication style, which may not resonate with all viewers. Despite these concerns, the overall educational value and audience engagement make this partnership a strong fit for Zero to Mastery.
